In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import caiman
# import wrappers.caiman_wrapper as cm
# import wrappers.suite2p_wrapper as s2p

from caiman.source_extraction.cnmf.params import CNMFParams
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.motion_correction import MotionCorrect

info = {}

In [ ]:
file_path = join_file_path(
    '/Users', 'shogoakiyama', 'caiman_data', 
    'example_movies', 'Sue_2x_3000_40_-46.tif')

In [ ]:
opts = CNMFParams()
motion_opts = opts.get_group('motion')

### motion correction

In [ ]:
mc = MotionCorrect(file_path, dview=None, **motion_opts)
mc.motion_correct(save_movie=True)

In [ ]:
fname_new = caiman.save_memmap(mc.mmap_file, base_name='memmap_', order='C',
                           border_to_0=0 if mc.border_nan is 'copy' else mc.border_to_0, dview=None) # exclude borders
Yr, dims, T = caiman.load_memmap(fname_new)
info['images'] = np.array(np.reshape(Yr.T, [T] + list(dims), order='F') )

In [ ]:
info['images'].shape

In [ ]:
plt.imshow(info['images'][0])

### CNMF

In [ ]:
cnm = cnmf.CNMF(1, params=opts, dview=None)
cnm = cnm.fit(info['images'])

In [ ]:
from caiman.utils.visualization import get_contours
from caiman.base.rois import com

fig, ax = plt.subplots()

# ax.pie([100, 200, 300, 400, 500])
ax.imshow(Cn, interpolation=None,
          vmin=np.percentile(Cn[~np.isnan(Cn)], 1),
          vmax=np.percentile(Cn[~np.isnan(Cn)], 99))

coordinates = get_contours(cnm.estimates.A, np.shape(Cn), 0.9, 'nrg', False)
coords = []
for x in coordinates:
    _c = []
    for y in x['coordinates']:
        _c.append(list(y))
    coords.append(_c)

coordinates = coords

# plot contours
for c in coordinates:
    plt.plot(*np.array(c).T, c='white')

# display number
d1, d2 = np.shape(Cn)
d, nr = np.shape(cnm.estimates.A)
cm = com(cnm.estimates.A, d1, d2)

max_number = cnm.estimates.A.shape[1]

for i in range(np.minimum(nr, max_number)):
    ax.text(cm[i, 1], cm[i, 0], str(i + 1), color=colors)

In [ ]:
info['images'] = Cn
info['coordinates'] = coordinates
info['cm'] = cm

In [ ]:
import caiman.utils.visualization as visualization

# get roi center
cont = visualization.get_contours(cnm.estimates.A, cnm.dims, thr=0.9, thr_method='nrg', swap_dim=False)
cont_cent = np.zeros([len(cont), 2])
for i in range(len(cont)):
    cont_cent[i, :] = np.nanmean(cont[i]['coordinates'], axis=0)

iscell = np.zeros(cont_cent.shape[0])
iscell[cnm.estimates.idx_components]=1

In [ ]:
info['fluo'] = cnm.estimates.C
info['iscell'] = iscell
info['roi'] = cont_cent

In [ ]:
# info['movies'] = cnm.estimates.play_movie(
#     info['images'], q_max=99.9, gain_res=2, magnification=2, 
#     bpx=info['border_to_0'], include_bck=False, display=False)

### component filtering

In [ ]:

# parameters for component evaluation
min_SNR = 1.5               # signal to noise ratio for accepting a component
rval_thr = 0.8              # space correlation threshold for accepting a component
min_cnn_thr = 0.99          # threshold for CNN based classifier
cnn_lowest = 0.05           # neurons with cnn probability lower than this value are rejected
#cnm_seeded.estimates.restore_discarded_components()
cnm.params.set('quality', {'min_SNR': min_SNR,
                           'rval_thr': rval_thr,
                           'use_cnn': True,
                           'min_cnn_thr': min_cnn_thr,
                           'cnn_lowest': cnn_lowest})

#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier

cnm.estimates.evaluate_components(info['images'], cnm.params, dview=None)

In [ ]:
cnm.estimates.plot_contours(img=Cn, idx=cnm.estimates.idx_components)

In [ ]:
from PIL import Image
img = Image.fromarray(np.uint8(info['images'][0]))

In [ ]:
pilImg